In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///../data/raw/JulenC.db')

In [3]:
df_business = pd.read_sql_table('business_info', engine)

In [4]:
df_personal = pd.read_sql_table('personal_info', engine)

In [5]:
df_rank = pd.read_sql_table('rank_info', engine)

In [6]:
df_business.head()

,id,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
0,3033,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
1,3381,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
2,2949,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
3,7380,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
4,6568,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5


In [7]:
df_business.sort_values("realTimePosition")

,id,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
0,3033,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
1,3381,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
2,2949,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
3,7380,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
4,6568,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5
...,...,...,...,...,...,...,...
2155,4524,2207,Technology ==> mobile gaming,1.0 BUSD,nan millions USD,NaN,2151
2154,5983,2206,Manufacturing ==> antennas,1.0 BUSD,nan millions USD,NaN,2151
2136,3259,2188,Technology ==> Stripe,1.0 BUSD,0.0 millions USD,NaN,2152
2133,2855,2185,Metals & Mining ==> metallurgy,1.0 BUSD,nan millions USD,NaN,2153


In [8]:
df_rank.head()

,id,name,position,Unnamed: 0
0,9141,anant ASAVABHOKHIN & FAmily,NaN,0
1,3602,araSH FERDOWSI,NaN,1
2,1750,banthoon laMSAM & FAMIly,NaN,2
3,5586,bill adderlEY & family,NaN,3
4,3870,boris ZINGAREVIch,NaN,4


In [9]:
df = df_business.merge(df_rank, on = ['id', 'Unnamed: 0'], how = 'left')

In [10]:
df = df.merge(df_personal, on = ['id', 'Unnamed: 0'], how = 'outer')

In [11]:
df.drop(['Unnamed: 0', 'lastName'], axis = 1, inplace = True)

In [12]:
df.name = df.name.str.title()

In [13]:
df.position = df.position.astype("int64")

In [14]:
df.worth = df.worth.str.replace('BUSD', '').astype('float64')

In [15]:
newcol = list(df.columns)
newcol[2] = "worth (BUSD)"
df.columns = newcol

In [16]:
df.age = df.age.str.replace(" years old", "")

In [17]:
df.age = df.age.fillna(value = np.nan).astype("float64")

In [18]:
df.age = df.age.apply(lambda x: 2019 - x if x > 1900 else x)

In [19]:
df["area"] = df["Source"].str.split("==>").str[0]

In [20]:
df["source"] = df["Source"].str.split("==>").str[1]

In [21]:
df.drop("Source", axis = 1, inplace = True)

In [22]:
newcol = list(df.columns)
newcol[2] = "worthChage (millions USD)"
df.columns = newcol

In [23]:
df["worthChage (millions USD)"] = df["worthChage (millions USD)"].str.replace(" millions USD", "")

In [24]:
df.loc[df["worthChage (millions USD)"] == "nan", "worthChage (millions USD)"] = np.nan

In [25]:
df["worthChage (millions USD)"] = df["worthChage (millions USD)"].astype("float64")

In [26]:
df.loc[df["gender"] == "Male", "gender"] = "M"

In [27]:
df.loc[df["gender"] == "Female", "gender"] = "F"

In [28]:
df.loc[df["gender"] == 'None', "gender"] = None

In [29]:
df.drop("realTimeWorth", axis = 1, inplace = True)

In [30]:
df.loc[df["country"] == 'None', "country"] = None

In [31]:
df.to_csv("../data/processed/forbes_clean.csv")

In [32]:
df.head()

,id,worth (BUSD),worthChage (millions USD),realTimePosition,name,position,age,gender,country,image,area,source
0,3033,112.0,0.000,1,Jeff Bezos,1,54.0,M,United States,https://specials-images.forbesimg.com/imageser...,Technology,Amazon
1,3381,90.0,-0.001,2,Bill Gates,2,62.0,M,United States,https://specials-images.forbesimg.com/imageser...,Technology,Microsoft
2,2949,84.0,-0.002,3,Warren Buffett,3,87.0,M,United States,https://specials-images.forbesimg.com/imageser...,Finance and Investments,Berkshire Hathaway
3,7380,72.0,0.000,4,Bernard Arnault,4,69.0,None,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail,LVMH
4,6568,71.0,0.000,5,Mark Zuckerberg,5,34.0,None,None,https://specials-images.forbesimg.com/imageser...,Technology,Facebook


In [33]:
df.isnull().sum()/len(df)*100

id                            0.000000
worth (BUSD)                  0.000000
worthChage (millions USD)     4.846014
realTimePosition              0.000000
name                          0.000000
position                      0.000000
age                           2.943841
gender                       25.135870
country                      62.952899
image                         0.000000
area                          0.000000
source                        0.000000
dtype: float64